

python trainer.py --db sleepedf --gpu 0 --from_fold 0 --to_fold 19


In [2]:
import warnings

warnings.filterwarnings("ignore")
import argparse
import importlib
import os

In [3]:
from train import train

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [21]:
from_fold = 0
to_fold = 19
random_seed = 42
db = "sleepedf"
suffix = ""

In [4]:
def run(args, db, gpu, from_fold, to_fold, suffix="", random_seed=42):
    # Set GPU visible

    # Config file
    config_file = os.path.join("config", f"{db}.py")
    spec = importlib.util.spec_from_file_location("*", config_file)
    config = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config)

    # Output directory
    output_dir = f"out_{db}{suffix}"

    assert from_fold <= to_fold
    assert to_fold < config.params["n_folds"]

    # Training
    for fold_idx in range(from_fold, to_fold + 1):
        train(
            args=args,
            config_file=config_file,
            fold_idx=fold_idx,
            output_dir=os.path.join(output_dir, "train"),
            log_file=os.path.join(output_dir, f"train_{gpu}.log"),
            restart=True,
            random_seed=random_seed + fold_idx,
        )

        # Reset tensorflow graph
        # tf.reset_default_graph()

In [15]:


config_file = os.path.join("config", f"{db}.py")
config_file

'config/sleepedf.py'

In [14]:
spec = importlib.util.spec_from_file_location("*", config_file)
spec

ModuleSpec(name='*', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f9a16688dd8>, origin='config/sleepedf.py')

In [23]:
output_dir = f"out_{db}{suffix}"
output_dir = os.path.join(output_dir, "train")
output_dir

'out_sleepedf/train'

In [5]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--db", type=str, required=True)
    parser.add_argument("--gpu", type=int, required=True)
    parser.add_argument("--from_fold", type=int, required=True)
    parser.add_argument("--to_fold", type=int, required=True)
    parser.add_argument("--suffix", type=str, default="")
    parser.add_argument("--random_seed", type=int, default=42)
    parser.add_argument("--test_seq_len", type=int, default=20)
    parser.add_argument("--test_batch_size", type=int, default=15)
    parser.add_argument("--n_epochs", type=int, default=200)
    args = parser.parse_args()

    run(
        args=args,
        db=args.db,
        gpu=args.gpu,
        from_fold=args.from_fold,
        to_fold=args.to_fold,
        suffix=args.suffix,
        random_seed=args.random_seed,
    )

usage: ipykernel_launcher.py [-h] --db DB --gpu GPU --from_fold FROM_FOLD
                             --to_fold TO_FOLD [--suffix SUFFIX]
                             [--random_seed RANDOM_SEED]
                             [--test_seq_len TEST_SEQ_LEN]
                             [--test_batch_size TEST_BATCH_SIZE]
                             [--n_epochs N_EPOCHS]
ipykernel_launcher.py: error: argument --from_fold: invalid int value: '/home/gs/.local/share/jupyter/runtime/kernel-v3ea9bd556030d15e172253508425798cb0891ef5a.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/gs/miniconda3/envs/tinysleepnet_env2/lib/python3.6/argparse.py", line 2294, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/gs/.local/share/jupyter/runtime/kernel-v3ea9bd556030d15e172253508425798cb0891ef5a.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gs/miniconda3/envs/tinysleepnet_env2/lib/python3.6/argparse.py", line 1766, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/home/gs/miniconda3/envs/tinysleepnet_env2/lib/python3.6/argparse.py", line 1972, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/home/gs/miniconda3/envs/tinysleepnet_env2/lib/python3.6/argparse.py", line 1912, in consume_optional
    take_action(action, args, option_string)
  File "/home/gs/miniconda3/envs/tinysleepnet_env2/lib/python3.6/argparse.py", line 1824

TypeError: object of type 'NoneType' has no len()